<a href="https://colab.research.google.com/github/kimdonggyu2008/hufs_hackerthon/blob/main/BBC_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#사전세팅


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install wandb

In [3]:
!pip install keras.preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.7 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import string
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

#import wandb
import os

In [5]:
# os.environ['WANDB_API_KEY']='513a1f0c050fa7f60a76b5232e904d8df397082e'
# os.environ['WANDB_ENTITY']='article classification'

In [6]:
# !wandb login

#감정분석(필요?)

In [7]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 31.8 MB/s eta 0:00:00


In [8]:
!apt-get update -qq
!apt-get install -qq fonts-nanum

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package fonts-nanum.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

In [10]:
#augmented, 500개로 맞춤
#augmented2, 1000개로 맞춤
#augmented_combined, 동일 카테고리별로 5개씩 묶고, 합친 갯수가 200개보다 적으면 200개까지 증강
#augmented_combined3, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 200개보다 적으면 50개까지 증강
#augmented_combined4, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 200개보다 적으면 50개까지 증강
#augmented_combined5, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 200개보다 적으면 50개까지 증강
#그리고 지역은 5000개까지 줄임
#augmented_combined5, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
df=pd.read_csv('/content/drive/MyDrive/data/dacon_project/data/train_augmented_combined10.csv')

In [11]:
okt=Okt()

In [12]:
def preprocess_korean(text):
    # 특수문자 제거 및 소문자 변환
    text = re.sub(r'\W+', ' ', text)
    # 형태소 분석 후 명사만 추출
    tokens = okt.nouns(text)
    return ' '.join(tokens)

#토크나이징 및 설정


In [13]:
# 카테고리 목록과 매핑될 레이블 번호를 생성합니다.
categories = [
    "지역", "경제:부동산", "사회:사건_사고", "경제:반도체", "사회:사회일반", "사회:교육_시험",
    "정치:국회_정당", "사회:의료_건강", "경제:취업_창업", "스포츠:올림픽_아시안게임",
    "경제:산업_기업", "문화:전시_공연", "경제:자동차", "경제:경제일반", "사회:장애인",
    "스포츠:골프", "정치:선거", "경제:유통", "IT_과학:모바일", "사회:여성",
    "사회:노동_복지", "사회:환경", "경제:서비스_쇼핑", "경제:무역", "정치:행정_자치",
    "국제", "문화:방송_연예", "스포츠:축구", "경제:금융_재테크", "정치:청와대",
    "문화:출판", "IT_과학:IT_과학일반", "IT_과학:인터넷_SNS", "문화:미술_건축",
    "정치:정치일반", "IT_과학:과학", "문화:문화일반", "문화:학술_문화재", "문화:요리_여행",
    "경제:자원", "문화:종교", "IT_과학:콘텐츠", "사회:미디어", "사회:날씨",
    "스포츠:농구_배구", "문화:음악", "문화:생활", "IT_과학:보안", "스포츠:월드컵",
    "경제:증권_증시", "정치:북한", "정치:외교", "스포츠:스포츠일반", "문화:영화",
    "스포츠:야구", "경제:외환"
]

# # DataFrame을 랜덤하게 섞습니다.
# shuffled = df.reindex(np.random.permutation(df.index))

# # 각 카테고리에 대해 데이터 샘플링
# num_of_categories = 45000
# sampled_data = []
# for category in categories:
#     sampled_data.append(shuffled[shuffled['분류'] == category][:num_of_categories])

# # 샘플링된 데이터를 하나의 데이터프레임으로 합칩니다.
# concated = pd.concat(sampled_data, ignore_index=True)

# # 데이터프레임을 다시 섞습니다.
# concated = concated.reindex(np.random.permutation(concated.index))

# # LABEL 열 생성 (카테고리별 레이블 부여)
# concated['라벨'] = concated['분류'].apply(lambda x: categories.index(x))

# # One-hot 인코딩
# labels = to_categorical(concated['라벨'], num_classes=len(categories))

# #카테고리 열 삭제
# if '분류' in concated.columns:
#     concated = concated.drop(['분류'], axis=1)

# print(concated['라벨'][:10])
# print(concated['분류'])
# print(labels[:10])


In [14]:

# # 기사 제목과 키워드를 결합합니다.
# df['text'] = df['제목'] + " " + df['키워드']

# # DataFrame을 랜덤하게 섞습니다.
# shuffled = df.reindex(np.random.permutation(df.index))

# # 각 카테고리에 대해 데이터 샘플링
# num_of_categories = 45000
# sampled_data = []
# for category in categories:
#     sampled_data.append(shuffled[shuffled['분류'] == category][:num_of_categories])

# # 샘플링된 데이터를 하나의 데이터프레임으로 합칩니다.
# concated = pd.concat(sampled_data, ignore_index=True)

# # 데이터프레임을 다시 섞습니다.
# concated = concated.reindex(np.random.permutation(concated.index))

# # LABEL 열 생성 (카테고리별 레이블 부여)
# concated['라벨'] = concated['분류'].apply(lambda x: categories.index(x))

# # One-hot 인코딩
# labels = to_categorical(concated['라벨'], num_classes=len(categories))

# text_lengths = concated['text'].apply(len)
# average_length = text_lengths.mean()

# # 결합된 텍스트 확인
# print(concated[['text', '라벨']].head())

In [15]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical

# 기사 제목과 키워드를 결합합니다.
df['text'] = df['제목'] + " " + df['키워드']

# 카테고리 목록 생성
categories = df['분류'].unique().tolist()

# DataFrame을 랜덤하게 섞습니다.
shuffled = df.sample(frac=1).reset_index(drop=True)

# 각 카테고리에 대해 데이터 샘플링
num_of_categories = 45000
sampled_data = []

for category in categories:
    category_data = shuffled[shuffled['분류'] == category]
    sampled_data.append(category_data[:min(len(category_data), num_of_categories)])

# 샘플링된 데이터를 하나의 데이터프레임으로 합칩니다.
concated = pd.concat(sampled_data, ignore_index=True)

# 데이터프레임을 다시 섞습니다.
concated = concated.sample(frac=1).reset_index(drop=True)

# LABEL 열 생성 (카테고리별 레이블 부여)
concated['라벨'] = concated['분류'].apply(lambda x: categories.index(x))

# One-hot 인코딩
labels = to_categorical(concated['라벨'], num_classes=len(categories))

# 텍스트 길이 계산 및 평균 길이 출력
text_lengths = concated['text'].apply(len)
average_length = text_lengths.mean()
print(f'결합된 텍스트의 평균 길이: {average_length}')

# 데이터 크기 확인 (x와 y의 길이가 일치해야 함)
print(f'Text data size: {len(concated["text"])}')
print(f'Label data size: {len(labels)}')

# 결합된 텍스트 및 라벨 확인
print(concated[['text', '라벨']].head())


결합된 텍스트의 평균 길이: 726.8032059984915
Text data size: 1126950
Label data size: 1126950
                                                text  라벨
0  경기농협 팜스테이마을 [창간 도내 특집] 추천 34 경기농협,추천,도내,팜스테이,마...  48
1  일본인, 선결제야” 왜 “숙박비 90여만원 격리 창밖으로 뿌려 숙박비,선결제,격리,...   2
2  추석맞이 가공식품 우수농산물과 지역 판매 용인특례시 우수농산물,용인,특례시,지역,우...   9
3  "푸바오 안녕" 포토] 배웅하는 강바오 송바오 [슬라이드 푸바오,안녕,배웅,강바오,...  44
4  모집 올해 수출상담회' '언택트 용인시 중진공 참여기업 경기지역본부 중진공,경기,지...  37


In [16]:
n_most_common_words = 30000
max_len = 300
tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(concated['제목'].values)
sequences = tokenizer.texts_to_sequences(concated['제목'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len)

Found 83625 unique tokens.


In [17]:
# n_most_common_words = 20000
tokenizer = Tokenizer(num_words=n_most_common_words)

# 기사 제목이나 내용을 토큰화할 텍스트로 설정
texts = df['text'].values  # '제목' 칼럼을 기준으로 사용

# 토큰화 및 텍스트 시퀀스 변환
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# 각 기사의 토큰 개수를 확인
token_counts = [len(seq) for seq in sequences]

# 토큰 수의 최대값 찾기
# max_len = sum(token_counts)/len(token_counts)
maxx_len = max(token_counts)
print(f"모든 기사에서 토큰 수의 최대값: {maxx_len}")

# 각 기사의 토큰 수 확인을 위한 데이터프레임 생성
token_count_df = pd.DataFrame({
    '기사 제목': texts,
    '토큰 수': token_counts
})

# 토큰 수 출력
print(token_count_df)
print(np.std(token_counts)*0.9)

모든 기사에서 토큰 수의 최대값: 2370
                                                     기사 제목  토큰 수
0        용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최 용인문화재단,인...    85
1        용인 농촌테마파크, 7~8월 단체체험객 체험료 지원 용인,농촌,테마파크,단체,체험객...    94
2        용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장 용인시,노후,주택,에너지,...    93
3        수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대 수원,용인,고양시,특례시,...   240
4        용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할 용인시,스페인,미국,국...   102
...                                                    ...   ...
1126945  로또당첨번호조회, 33억씩'(종합) '1등 당첨번호 당첨지역 7명 공개 932회 로...    93
1126946  용인 재계약 하나 '수상골프연습장' 수상골프연습장,재계약,전경,기흥호수,수상,골프,...   244
1126947  낸 재정위원회 김진영 KBL, 회부[공식발표] 음주사고 삼성 KBL,음주사고,삼성,...    78
1126948  용인시장 [제21회 용인시 경인일보배 인터뷰|이상일 배드민턴] 용인시,경인일보배,배...    99
1126949  [부고] 중구청장)씨 빙부상 김길성(서울 김길성,서울,중구청장,빙부상,최원락,별세,...    27

[1126950 rows x 2 columns]
128.60072945587964


In [18]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [19]:
# ReduceLROnPlateau 콜백을 사용해 학습률 감소
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7, verbose=1)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.25)

In [21]:
import tensorflow as tf
from transformers import TFGPT2Model

class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]  # CLS 토큰에 해당하는 마지막 위치의 출력을 사용
        prediction = self.classifier(cls_token)
        return prediction


In [22]:

# 모델 설정
model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2", num_labels=56)  # 분류 클래스 수를 56으로 설정
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.CategoricalCrossentropy()  # 원-핫 인코딩 레이블에 적합한 손실 함수


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.2.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [23]:
checkpoint_path = "/content/drive/MyDrive/data/dacon_project/model/best_gpt2_model_checkpoint.keras"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', save_best_only=True, verbose=1)


In [24]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=1)


In [25]:

# 모델 컴파일
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


In [26]:
model.load_weights(checkpoint_path)



/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [28]:
from tensorflow.keras.models import load_model

In [34]:
checkpoint_dir = "/content/drive/MyDrive/data/dacon_project/model"


In [35]:
# from transformers import TFGPT2ForSequenceClassification, AutoTokenizer

# # 저장된 모델과 토크나이저 불러오기
# model = TFGPT2ForSequenceClassification.from_pretrained(checkpoint_dir)
# tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)


OSError: /content/drive/MyDrive/data/dacon_project/model does not appear to have a file named config.json. Checkout 'https://huggingface.co//content/drive/MyDrive/data/dacon_project/model/tree/main' for available files.

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=30,
                    validation_data=(X_test, y_test),
                    callbacks=[checkpoint, early_stopping, reduce_lr])

Epoch 1/30
   2/6604 ━━━━━━━━━━━━━━━━━━━━ 3:34:15 2s/step - accuracy: 0.0195 - loss: 5.8920   

In [29]:
model = load_model(checkpoint_path)


TypeError: Could not locate class 'TFGPT2ForSequenceClassification'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': None, 'class_name': 'TFGPT2ForSequenceClassification', 'config': {'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}, 'registered_name': 'TFGPT2ForSequenceClassification', 'build_config': {'input_shape': [None, 300]}, 'compile_config': {'optimizer': {'module': 'keras.optimizers', 'class_name': 'Adam', 'config': {'name': 'adam', 'learning_rate': 0.0010000000474974513, 'weight_decay': None, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}, 'registered_name': None}, 'loss': {'module': 'keras.losses', 'clas

In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Best model loss: {loss}, accuracy: {accuracy}')

In [ ]:


# 학습 결과 시각화
import matplotlib.pyplot as plt

epochs = range(1, len(history.history['accuracy']) + 1)
plt.plot(epochs, history.history['loss'], label='Training loss')
plt.plot(epochs, history.history['val_loss'], label='Validation loss')
plt.plot(epochs, history.history['accuracy'], label='Training accuracy')
plt.plot(epochs, history.history['val_accuracy'], label='Validation accuracy')
plt.title('Model Loss and Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.legend()
plt.show()


#모델


In [ ]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, Embedding, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D
# from tensorflow.keras.layers import Add
# from tensorflow.keras.optimizers import AdamW
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.layers import Embedding, LSTM
# import tensorflow as tf
# from tensorflow.keras.callbacks import ReduceLROnPlateau
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# from sklearn.metrics import f1_score
# import numpy as np




In [ ]:
# import matplotlib.pyplot as plt

# # 학습 결과 시각화
# def plot_training_history(history):
#     # 정확도 그래프
#     plt.plot(history.history['accuracy'], label='Train Accuracy')
#     plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
#     plt.title('Accuracy')
#     plt.xlabel('Epochs')
#     plt.ylabel('Accuracy')
#     plt.legend()
#     plt.show()

#     # 손실 그래프
#     plt.plot(history.history['loss'], label='Train Loss')
#     plt.plot(history.history['val_loss'], label='Validation Loss')
#     plt.title('Loss')
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.show()

In [ ]:
# # 기본 설정
# n_most_common_words = 70000  # 어휘 수 80,000개로 설정
# emb_dim = 128  # 임베딩 벡터의 차원 설정
# #max_len = X.shape[1]  # 시퀀스의 길이 (기사 제목의 길이)
# num_heads = 8  # Multi-head attention에서 헤드의 개수
# ff_dim = 128  # Feed Forward Network의 출력 차원
# num_classes = 56  # 최종 클래스의 개수
# hidden_units=128

# # 최적화된 하이퍼파라미터
# learning_rate = 0.00005        # 학습률 (조정 가능)
# batch_size = 32             # 배치 크기
# epochs = 20                 # 최대 에포크 수
# dropout_rate = 0.7           # 드롭아웃 비율
# l2_lambda = 0.001             # L2 정규화 값
# patience = 7                  # 조기 종료를 위한 patience
# min_delta = 0.001             # 조기 종료를 위한 최소 손실 감소량
# validation_split = 0.2        # 검증 데이터 비율


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.25)

In [ ]:
# def reset_weights(model):
#     # 모델의 모든 레이어의 가중치를 초기화
#     for layer in model.layers:
#         if hasattr(layer, 'kernel_initializer') and layer.kernel_initializer:
#             layer.kernel.assign(layer.kernel_initializer(layer.kernel.shape))
#         if hasattr(layer, 'bias_initializer') and layer.bias_initializer:
#             layer.bias.assign(layer.bias_initializer(layer.bias.shape))

In [ ]:
# optimizer=AdamW(learning_rate=learning_rate)


In [ ]:
# model = Sequential()


# model.add(Embedding(input_dim=n_most_common_words, output_dim=emb_dim))
# model.add(LSTM(hidden_units))
# model.add(Dense(num_classes, activation='softmax'))

# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model.summary()

In [ ]:
# # 커스텀 F1 스코어 메트릭 정의 (텐서 기반)
# class F1Score(tf.keras.metrics.Metric):
#     def __init__(self, name="f1_score", **kwargs):
#         super(F1Score, self).__init__(name=name, **kwargs)
#         self.precision = tf.keras.metrics.Precision()
#         self.recall = tf.keras.metrics.Recall()

#     def update_state(self, y_true, y_pred, sample_weight=None):
#         y_pred = tf.argmax(y_pred, axis=1)
#         y_true = tf.argmax(y_true, axis=1)
#         self.precision.update_state(y_true, y_pred, sample_weight)
#         self.recall.update_state(y_true, y_pred, sample_weight)

#     def result(self):
#         precision = self.precision.result()
#         recall = self.recall.result()
#         return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

#     def reset_states(self):
#         self.precision.reset_states()
#         self.recall.reset_states()


In [ ]:
# # 콜백 함수 설정
# es = EarlyStopping(
#     monitor='val_loss',     # 검증 손실을 모니터링
#     mode='min',             # 'min' 모드로 설정하여, 손실이 감소하는 것을 추적
#     patience=3,             # 손실이 더 이상 개선되지 않는 즉시 학습 중단
#     restore_best_weights=True  # 가장 성능이 좋았던 가중치로 복원
# )
# mc = ModelCheckpoint('best_model.keras', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)


In [ ]:
# # 모델 컴파일 (F1 스코어를 메트릭으로 추가)
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', F1Score()])


In [ ]:
# # 모델 학습
# history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
#                     callbacks=[es, mc], validation_data=(X_test, y_test))

In [ ]:
# plot_training_history(history)

#시행착오


In [ ]:
# emb_dim = 128
# # 입력층
# input_layer = Input(shape=(max_len,))

# # Embedding 레이어
# embedding_layer = Embedding(input_dim=n_most_common_words, output_dim=emb_dim, input_length=max_len)(input_layer)


In [ ]:

# # 트랜스포머 블록 정의
# def transformer_block(x, head_size, num_heads, ff_dim, dropout=0.3):
#     # Multi-Head Attention
#     attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
#     attention_output = Dropout(dropout)(attention_output)

#     # Add & Normalize
#     attention_output = Add()([x, attention_output])
#     attention_output = LayerNormalization()(attention_output)

#     # Feed Forward Network
#     ffn_output = Dense(ff_dim, activation='relu',kernel_regularizer=l2(0.001))(attention_output)
#     ffn_output = Dropout(dropout)(ffn_output)
#     ffn_output = Dense(emb_dim)(ffn_output)

#     # Add & Normalize
#     ffn_output = Add()([attention_output, ffn_output])
#     ffn_output = LayerNormalization()(ffn_output)

#     return ffn_output


In [ ]:
# # 모델 빌드 함수
# def build_model(n_most_common_words, emb_dim, max_len, num_heads, ff_dim, num_classes, dropout_rate):
#     # 입력층
#     input_layer = Input(shape=(max_len,))

#     # Embedding 레이어
#     embedding_layer = Embedding(input_dim=n_most_common_words, output_dim=emb_dim, input_length=max_len)(input_layer)

#     # 트랜스포머 블록 적용
#     x = transformer_block(embedding_layer, emb_dim, num_heads, ff_dim, dropout=dropout_rate)

#     # Global Average Pooling
#     x = GlobalAveragePooling1D()(x)

#     # Dropout (최종 분류 전에 추가)
#     x = Dropout(dropout_rate)(x)

#     # 분류를 위한 Dense 레이어 (num_classes = 56)
#     output_layer = Dense(num_classes, activation='softmax')(x)

#     # 모델 정의
#     model = Model(inputs=input_layer, outputs=output_layer)

#     return model

In [ ]:
# model = build_model(n_most_common_words=70000,
#                     emb_dim=128,
#                     max_len=max_len,
#                     num_heads=8,
#                     ff_dim=128,
#                     num_classes=56,
#                     dropout_rate=0.3)

In [ ]:
# import numpy as np
# import tensorflow as tf
# import random

# # 시드 고정
# seed = 42
# np.random.seed(seed)
# tf.random.set_seed(seed)
# random.seed(seed)


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.25)

In [ ]:
# early_stopping = EarlyStopping(
#     monitor='val_loss',     # 검증 손실을 모니터링
#     mode='min',             # 'min' 모드로 설정하여, 손실이 감소하는 것을 추적
#     patience=3,             # 손실이 더 이상 개선되지 않는 즉시 학습 중단
#     restore_best_weights=True  # 가장 성능이 좋았던 가중치로 복원
# )

In [ ]:
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

In [ ]:
# epochs=[10,15]
# learning_rates=[0.0005,0.0008,0.001]
# batch_sizes=[64,128,256]
# dropout_rates=[0.3,0.5,0.75]

# best_val_loss = np.inf  # 현재 최적의 검증 손실 값
# best_val_acc=0
# best_hyperparams = {}
# best_model_weights = None

# for epoch in epochs:
#   for learning_rate in learning_rates:
#     for batch_size in batch_sizes:
#       for dropout_rate in dropout_rates:

#         print(f"Recent epoch: {epoch}")
#         print(f"Recent learning_rate: {learning_rate}")
#         print(f"Recent batch_size: {batch_size}")
#         print(f"Recent dropout_rate: {dropout_rate}")

#         model = build_model(n_most_common_words=n_most_common_words,
#                     emb_dim=emb_dim,
#                     max_len=max_len,
#                     num_heads=num_heads,
#                     ff_dim=ff_dim,
#                     num_classes=num_classes,
#                     dropout_rate=dropout_rate)

#         reset_weights(model)

#         optimizer=AdamW(learning_rate=learning_rate)
#         model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#         checkpoint = ModelCheckpoint(
#                     filepath=f'/content/drive/MyDrive/data/dacon_project/model/model_epoch{epoch}_lr{learning_rate}_bs{batch_size}_dr{dropout_rate}.keras',  # 모델을 저장할 경로
#                     monitor='val_loss',     # val_loss를 모니터링
#                     mode='min',             # val_loss가 최소값일 때 저장
#                     save_best_only=True,    # 최적의 val_loss 갱신 시에만 저장
#                     verbose=1               # 모델 저장 시 메시지 출력
#                 )

#         early_stopping = EarlyStopping(
#         monitor='val_loss',     # 검증 손실을 모니터링
#         mode='min',             # 'min' 모드로 설정하여, 손실이 감소하는 것을 추적
#         patience=3,             # 손실이 더 이상 개선되지 않는 즉시 학습 중단
#         restore_best_weights=True  # 가장 성능이 좋았던 가중치로 복원
#         )

#         history = model.fit(
#         X_train,
#         y_train,
#         epochs=epoch,
#         batch_size=batch_size,
#         validation_split=validation_split,
#         callbacks=[early_stopping, checkpoint],
#         verbose=1)


#         val_loss = min(history.history['val_loss'])
#         val_acc = max(history.history['val_accuracy'])
#         if (val_loss < best_val_loss and val_acc > best_val_acc) and val_loss < 1:

#                     best_val_loss = val_loss
#                     best_val_acc = val_acc
#                     best_hyperparams['epoch'] = epoch
#                     best_hyperparams['learning_rate'] = learning_rate
#                     best_hyperparams['batch_size'] = batch_size
#                     best_hyperparams['dropout_rate'] = dropout_rate
#                     best_model_weights=model.get_weights()

#         #plot_training_history(history)

# # 최종 결과 출력
# model.save('/content/drive/MyDrive/data/dacon_project/model/best_model.keras')

In [ ]:
# model.set_weights(best_model_weights)

In [ ]:
# from tensorflow.keras.models import load_model

In [ ]:
# model=load_model('/content/drive/MyDrive/data/dacon_project/model/model_epoch10_lr0.0001_bs64_dr0.3.h5')

In [ ]:
# print(f"Best val_loss: {best_val_loss}")
# print(f"Best val_acc: {best_val_acc}")
# print(f"Best Hyperparameters: Epoch: {best_hyperparams['epoch']}, "
#       f"Learning Rate: {best_hyperparams['learning_rate']}, "
#       f"Batch Size: {best_hyperparams['batch_size']}, "
#       f"Dropout Rate: {best_hyperparams['dropout_rate']}")

In [ ]:
# # # 최적화된 하이퍼파라미터
# learning_rate = 0.001        # 학습률 (조정 가능)
# batch_size = 32             # 배치 크기
# epochs = 15                  # 최대 에포크 수
# dropout_rate = 0.7           # 드롭아웃 비율
# l2_lambda = 0.001             # L2 정규화 값
# patience = 5                  # 조기 종료를 위한 patience
# min_delta = 0.001             # 조기 종료를 위한 최소 손실 감소량
# validation_split = 0.2        # 검증 데이터 비율


In [ ]:
# from tensorflow.keras.optimizers import SGD

In [ ]:
# # SGD 옵티마이저로 변경
# optimizer = SGD(learning_rate=learning_rate)  # 필요시 momentum=0.9 추가 가능

# # 모델 컴파일
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# optimizer=AdamW(learning_rate=learning_rate)
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# reset_weights(model)
# history = model.fit(
#     X_train,
#     y_train,
#     epochs=epochs,
#     batch_size=batch_size,
#     validation_split=validation_split,
#     callbacks=[early_stopping])

In [ ]:
# plot_training_history(history)

#저장 및 정확도


In [ ]:
# 모델 전체를 저장하는 코드 (모델 아키텍처, 가중치, 컴파일 정보 포함)
model.save('/content/drive/MyDrive/data/dacon_project/result/bbc_edit_model.keras')  # 모델을 'model.h5' 파일로 저장

print("모델이 'bbc_edit_model.keras'로 저장되었습니다.")


In [ ]:
# accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
# accr = model.evaluate(X_test,y_test)
# print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
# txt = ["용인특례시, 가을 신학기 학교 급식 공급업체 위생점검"]
# seq = tokenizer.texts_to_sequences(txt)
# padded = pad_sequences(seq, maxlen=max_len)
# pred = model.predict(padded)
# labels = [
#     "지역", "경제:부동산", "사회:사건_사고", "경제:반도체", "사회:사회일반", "사회:교육_시험",
#     "정치:국회_정당", "사회:의료_건강", "경제:취업_창업", "스포츠:올림픽_아시안게임",
#     "경제:산업_기업", "문화:전시_공연", "경제:자동차", "경제:경제일반", "사회:장애인",
#     "스포츠:골프", "정치:선거", "경제:유통", "IT_과학:모바일", "사회:여성",
#     "사회:노동_복지", "사회:환경", "경제:서비스_쇼핑", "경제:무역", "정치:행정_자치",
#     "국제", "문화:방송_연예", "스포츠:축구", "경제:금융_재테크", "정치:청와대",
#     "문화:출판", "IT_과학:IT_과학일반", "IT_과학:인터넷_SNS", "문화:미술_건축",
#     "정치:정치일반", "IT_과학:과학", "문화:문화일반", "문화:학술_문화재", "문화:요리_여행",
#     "경제:자원", "문화:종교", "IT_과학:콘텐츠", "사회:미디어", "사회:날씨",
#     "스포츠:농구_배구", "문화:음악", "문화:생활", "IT_과학:보안", "스포츠:월드컵",
#     "경제:증권_증시", "정치:북한", "정치:외교", "스포츠:스포츠일반", "문화:영화",
#     "스포츠:야구", "경제:외환"
# ]
# # print(pred, labels[np.argmax(pred)])

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. test.csv 파일 로드
test_df = pd.read_csv('/content/drive/MyDrive/data/dacon_project/data/test.csv')

# 2. 데이터 전처리: 제목을 시퀀스로 변환
# 학습 시 사용했던 tokenizer를 사용하여 제목을 시퀀스로 변환
sequences_test = tokenizer.texts_to_sequences(test_df['제목'].values)

# 3. 시퀀스의 길이를 동일하게 맞추기 (max_len에 맞춰 패딩)
X_test = pad_sequences(sequences_test, maxlen=max_len)

# 4. 모델을 사용하여 예측
predictions = model.predict(X_test)

# 5. 예측 결과에서 가장 확률이 높은 클래스를 추출
predicted_labels = predictions.argmax(axis=1)
predicted_labels[predicted_labels>45]=0

predicted_class_names=[categories[label] for label in predicted_labels]

# 6. submission 데이터프레임 생성 (id와 예측된 레이블을 저장)
submission_df = pd.DataFrame({
    'ID': test_df['ID'],
    '분류': predicted_class_names
})

# 7. submission.csv 파일로 저장
submission_df.to_csv('/content/drive/MyDrive/data/dacon_project/result/submission.csv', index=False)

print("Submission 파일이 생성되었습니다.")


In [ ]:
df_test=pd.read_csv('/content/drive/MyDrive/data/dacon_project/data/test.csv')
df_result=pd.read_csv('/content/drive/MyDrive/data/dacon_project/result/submission.csv')
print("Shpe of test Data",df_test.shape)
print("Shpe of test Data",df_result.shape)
# df_test.head(30)

In [ ]:
df_result.head(30)

In [ ]:
print(df_result['분류'].value_counts())


In [ ]:
print(df_result['분류'])